In [1]:
import sys
from pathlib import Path
sys.path.append((Path.cwd().parent.parent).__str__())

In [4]:
import sympy as sp
from pyqpanda_alg.QAOA import qaoa
from pyqpanda3.core import CPUQVM, QProg, H

In [3]:
vars = sp.symbols('x0:3')
f = 2*vars[0]*vars[1] + 3*vars[2] - 1
print(qaoa.problem_to_z_operator(f))
operator = qaoa.problem_to_z_operator(f)
prog = QProg(3)
qubits = prog.qubits()
circuit = qaoa.pauli_z_operator_to_circuit(operator, qubits, 1)[0]
print(circuit)
print(circuit.originir())

{ qbit_total = 3, pauli_with_coef_s = { '':1 + 0j, 'Z2 ':-1.5 + 0j, 'Z1 ':-0.5 + 0j, 'Z0 ':-0.5 + 0j, 'Z0 Z1 ':0.5 + 0j, } }

          ┌───────────────┐                                ┌─┐ 
q_0:  |0>─┤RZ(-1.00000000)├ ───*── ──────────────── ───*── ┤I├ 
          ├───────────────┤ ┌──┴─┐ ┌──────────────┐ ┌──┴─┐ ├─┤ 
q_1:  |0>─┤RZ(-1.00000000)├ ┤CNOT├ ┤RZ(1.00000000)├ ┤CNOT├ ┤I├ 
          ├───────────────┤ ├─┬──┘ └──────────────┘ └────┘ └─┘ 
q_2:  |0>─┤RZ(-3.00000000)├ ┤I├─── ──────────────── ────── ─── 
          └───────────────┘ └─┘                                
 c :   / ═
          


QINIT 3
CREG 1
RZ q[2],(-3.00000000)
RZ q[1],(-1.00000000)
RZ q[0],(-1.00000000)
CNOT q[0],q[1]
RZ q[1],(1.00000000)
CNOT q[0],q[1]
I q[0]
I q[1]
I q[2]



In [20]:
import numpy as np
prog = QProg()
for q in qubits:
    prog << H(q)
prog << circuit
machine = CPUQVM()
machine.run(prog, shots=1)
qlist = machine.result().get_state_vector()
theory_func = sp.lambdify(vars, f, 'numpy')
for i in range(2**len(qubits)):
    key = np.binary_repr(i, len(qubits))[::-1]
    phase = -np.angle(qlist[i])
    theory = theory_func(*np.array([int(i) for i in key])) 
    theory_phase = 1 * theory
    print(key, phase, theory_phase)
    # 相差一个全局相位是正常的

000 -2.0 -1
100 -2.0 -1
010 -2.0 -1
110 -0.0 1
001 1.0 2
101 1.0000000000000002 2
011 1.0 2
111 3.0 4
